In [1]:
import sys
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import shapash
from shapash.data.data_loader import data_loading


In [2]:
from shapash.explainer.smart_explainer import SmartExplainer

In [3]:
data, dic = data_loading('titanic')
Xprepro = data.drop(['Survived'], axis = 1)
y = data.Survived

In [4]:
varcol = Xprepro.select_dtypes(object).columns.tolist()

In [5]:
categ_encod = OrdinalEncoder(cols = varcol, return_df=True).fit(Xprepro)

In [6]:
X = categ_encod.transform(Xprepro)

In [7]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2)

In [8]:
model = RandomForestClassifier()

In [9]:
model.fit(Xtrain, ytrain)

RandomForestClassifier()

In [10]:
ypred = pd.DataFrame(model.predict(Xtest), index= Xtest.index)
ypred_proba = pd.DataFrame(model.predict_proba(Xtest), index = Xtest.index)

In [11]:
xpl = SmartExplainer(features_dict=dic)

In [35]:
postprocess = {'Pclass' : {'type' : 'prefix', 'rule': 'Class type : '},
               'Fare' : {'type': 'suffix', 'rule': '$'},
              3 : {'type' : 'suffix', 'rule' : ' yo'},
              'Sex' : {'type': 'transcoding', 'rule':{'male': 'Homme', 'female':'Femme'}},
              'Port of embarkation': {'type' : 'case', 'rule': 'upper'}}

In [36]:
xpl.compile(Xtest, model, preprocessing=categ_encod, postprocessing=postprocess)

Backend: Shap TreeExplainer


In [37]:
xpl.x_pred

,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title
PassengerId,,,,,,,,,
147,Class type : Third class,"Andersson August Edvard (""Wennerstrom"")",Homme,27.0 yo,0,0,7.8$,SOUTHAMPTON,Mr
545,Class type : First class,Douglas Walter Donald,Homme,50.0 yo,1,0,106.42$,CHERBOURG,Mr
701,Class type : First class,Astor John Jacob (Madeleine Talmadge Force),Femme,18.0 yo,1,0,227.52$,CHERBOURG,Mrs
633,Class type : First class,Stahelin-Maeglin Max,Homme,32.0 yo,0,0,30.5$,CHERBOURG,Dr
886,Class type : Third class,Rice William (Margaret Norton),Femme,39.0 yo,0,5,29.12$,QUEENSTOWN,Mrs
...,...,...,...,...,...,...,...,...,...
150,Class type : Second class,Byles Thomas Roussel Davids,Homme,42.0 yo,0,0,13.0$,SOUTHAMPTON,Rev
839,Class type : Third class,Chip Chang,Homme,32.0 yo,0,0,56.5$,SOUTHAMPTON,Mr
24,Class type : First class,Sloper William Thompson,Homme,28.0 yo,0,0,35.5$,SOUTHAMPTON,Mr


In [16]:
xpl.plot.local_plot(index=27)

In [17]:
xpl.data['x_sorted'][0]

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8
PassengerId,,,,,,,,,
147,Mr,Homme,7.8,"Andersson August Edvard (""Wennerstrom"")",Class type : Third class,SOUTHAMPTON,27.0 yo,0,0
545,106.42,Mr,Homme,Class type : First class,Douglas Walter Donald,1,CHERBOURG,50.0 yo,0
701,Mrs,Femme,Class type : First class,227.52,CHERBOURG,Astor John Jacob (Madeleine Talmadge Force),0,1,18.0 yo
633,Homme,Class type : First class,Stahelin-Maeglin Max,Dr,30.5,CHERBOURG,32.0 yo,0,0
886,Class type : Third class,5,Mrs,Femme,39.0 yo,QUEENSTOWN,Rice William (Margaret Norton),29.12,0
...,...,...,...,...,...,...,...,...,...
150,Homme,Byles Thomas Roussel Davids,0,Rev,42.0 yo,SOUTHAMPTON,0,Class type : Second class,13
839,56.5,Chip Chang,Mr,Homme,32.0 yo,Class type : Third class,SOUTHAMPTON,0,0
24,Mr,Homme,Sloper William Thompson,35.5,Class type : First class,28.0 yo,0,SOUTHAMPTON,0


In [18]:
xpl.plot.compare_plot(row_num=[0, 1, 2])

In [19]:
xpl.plot.contribution_plot('Age')

In [40]:
xpl.inv_columns_dict

{'Pclass': 0,
 'Name': 1,
 'Sex': 2,
 'Age': 3,
 'SibSp': 4,
 'Parch': 5,
 'Fare': 6,
 'Embarked': 7,
 'Title': 8}

In [41]:
xpl.inv_features_dict

{'PassengerID': 'PassengerID',
 'Has survived ?': 'Survival',
 'Ticket class': 'Pclass',
 'Name, First name': 'Name',
 'Sex': 'Sex',
 'Age': 'Age',
 'Relatives such as brother or wife': 'SibSp',
 'Relatives like children or parents': 'Parch',
 'Passenger fare': 'Fare',
 'Port of embarkation': 'Embarked',
 'Title of passenger': 'Title'}

In [42]:
postprocess

{'Pclass': {'type': 'prefix', 'rule': 'Class type : '},
 'Age': {'type': 'suffix', 'rule': ' yo'},
 'Sex': {'type': 'transcoding', 'rule': {'male': 'Homme', 'female': 'Femme'}},
 'Embarked': {'type': 'case', 'rule': 'upper'}}

In [45]:
list(postprocess.keys())

['Pclass', 'Age', 'Sex', 'Embarked']

In [47]:
xpl.features_dict

{'PassengerID': 'PassengerID',
 'Survival': 'Has survived ?',
 'Pclass': 'Ticket class',
 'Name': 'Name, First name',
 'Sex': 'Sex',
 'Age': 'Age',
 'SibSp': 'Relatives such as brother or wife',
 'Parch': 'Relatives like children or parents',
 'Fare': 'Passenger fare',
 'Embarked': 'Port of embarkation',
 'Title': 'Title of passenger'}

In [64]:
a = list(map(xpl.features_dict.get, postprocess.keys()))

In [65]:
a

['Ticket class', 'Age', 'Sex', 'Port of embarkation']

In [69]:
postprocess

{'Pclass': {'type': 'prefix', 'rule': 'Class type : '},
 'Age': {'type': 'suffix', 'rule': ' yo'},
 'Sex': {'type': 'transcoding', 'rule': {'male': 'Homme', 'female': 'Femme'}},
 'Embarked': {'type': 'case', 'rule': 'upper'}}

In [101]:
test_dic = {'Pclass': {'type': 'prefix', 'rule': 'Class type : '},
 3: {'type': 'suffix', 'rule': ' yo'},
 'Sex': {'type': 'transcoding', 'rule': {'male': 'Homme', 'female': 'Femme'}},
 'Port of embarkation': {'type': 'case', 'rule': 'upper'}}

In [90]:
xpl.columns_dict

{0: 'Pclass',
 1: 'Name',
 2: 'Sex',
 3: 'Age',
 4: 'SibSp',
 5: 'Parch',
 6: 'Fare',
 7: 'Embarked',
 8: 'Title'}

In [71]:
b = list(map(xpl.inv_features_dict.get, test_dic.keys()))

In [73]:
test_dic['test'] = test_dic.pop('Age')

In [91]:
test_dic

{'Pclass': {'type': 'prefix', 'rule': 'Class type : '},
 3: {'type': 'suffix', 'rule': ' yo'},
 'Sex': {'type': 'transcoding', 'rule': {'male': 'Homme', 'female': 'Femme'}},
 'Port of embarkation': {'type': 'case', 'rule': 'upper'}}

In [86]:
test1 = list(map(xpl.inv_features_dict.get, test_dic.keys()))
test2 = list(map(xpl.columns_dict.get, test_dic.keys()))

In [87]:
test1

[None, None, 'Sex', 'Embarked']

In [100]:
test_dic

{'Pclass': {'type': 'prefix', 'rule': 'Class type : '},
 'Age': {'type': 'suffix', 'rule': ' yo'}}

In [102]:
new_dic = dict()
for i, key in enumerate(test_dic.keys()):
    if key in xpl.columns_dict.keys():
        new_dic[xpl.columns_dict[key]] = test_dic[key]
        
    elif key in xpl.inv_features_dict:
        new_dic[xpl.inv_features_dict[key]] = test_dic[key]
    else:
        new_dic[key] = test_dic[key]
        
    
    
    

In [103]:
new_dic

{'Pclass': {'type': 'prefix', 'rule': 'Class type : '},
 'Age': {'type': 'suffix', 'rule': ' yo'},
 'Sex': {'type': 'transcoding', 'rule': {'male': 'Homme', 'female': 'Femme'}},
 'Embarked': {'type': 'case', 'rule': 'upper'}}

In [104]:
postprocess

{'Pclass': {'type': 'prefix', 'rule': 'Class type : '},
 'Age': {'type': 'suffix', 'rule': ' yo'},
 'Sex': {'type': 'transcoding', 'rule': {'male': 'Homme', 'female': 'Femme'}},
 'Embarked': {'type': 'case', 'rule': 'upper'}}